# Cramer GAN implementation by tensorflow

In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility func

In [2]:
# save metrics
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics"
    if not os.path.isdir(path):
        os.makedirs(path)

    # save metrics
    plt.figure(figsize=(10,8))
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [3]:
# plot images
def save_imgs(images, plot_dim=(5,12), size=(12,5), epoch=None):
    # make directory if there is not
    path = "generated_figures"
    if not os.path.isdir(path):
        os.makedirs(path)

    num_examples = plot_dim[0]*plot_dim[1]
    num_examples = 60
    fig = plt.figure(figsize=size)

    for i in range(num_examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((96, 96, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()

# model

In [5]:
class Generator_UNet:
    def __init__(self, name="test"):
        self.reuse = False
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 96*96*3
        self.z_dim = 100
        self.name = name

    def __call__(self, inputs, gf_dim=64):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        def instance_norm(input, name):
            with tf.variable_scope(name):
                depth = input.get_shape()[3]
                scale = tf.get_variable("scale", [depth], initializer=tf.random_normal_initializer(1.0, 0.02, dtype=tf.float32))
                offset = tf.get_variable("offset", [depth], initializer=tf.constant_initializer(0.0))
                mean, variance = tf.nn.moments(input, axes=[1,2], keep_dims=True)
                epsilon = 1e-5
                inv = tf.rsqrt(variance + epsilon)
                normalized = (input-mean)*inv
            return scale*normalized + offset

        with tf.variable_scope('g_' + self.name, reuse=self.reuse):
            # 256x256x3
            print("called... name of g:" + str(self.name))
            inputs = tf.reshape(inputs, [-1, self.z_dim])
            fc1 = inputs
            fc2 = tf.layers.dense(fc1, 96*96*3)
            fc2 = tf.reshape(fc2, [-1, 96,96,3])


            e0 = tf.layers.conv2d(fc2, gf_dim, [3,3], [2,2], padding="SAME") # 48x48
            e1 = leaky_relu(e0)
            e1 = tf.layers.conv2d(e1, gf_dim*2, [3,3], [2,2], padding="SAME")
            #e1 = instance_norm(e1, name="e1")
            # 24x24

            e2 = leaky_relu(e1)
            e2 = tf.layers.conv2d(e2, gf_dim*4, [3,3], [2,2], padding="SAME")
            #e2 = instance_norm(e2, name="e2")
            # 12x12

            e3 = leaky_relu(e2)
            e3 = tf.layers.conv2d(e3, gf_dim*8, [3,3], [2,2], padding="SAME")
            #e3 = instance_norm(e3, name="e3")
            # 6x6

            e4 = leaky_relu(e3)
            e4 = tf.layers.conv2d(e4, gf_dim*8, [3,3], [2,2], padding="SAME")
            #e4 = instance_norm(e4, name="e4")
            # 3x3

            # 
            d0 = tf.nn.relu(e4)
            d0 = tf.layers.conv2d_transpose(d0, gf_dim*8, [3,3], [2,2], padding="SAME") 
            #d0 = instance_norm(d0, name="d0")
            #d0 = tf.nn.dropout(d0, 0.5)
            d0 = tf.concat([d0, e3], 3)
            # 6x6
            
            d1 = tf.nn.relu(d0)
            d1 = tf.layers.conv2d_transpose(d1, gf_dim*4, [3,3], [2,2], padding="SAME") 
            #d1 = instance_norm(d1, name="d1")
            #d1 = tf.nn.dropout(d1, 0.5)
            d1 = tf.concat([d1, e2], 3)
            # 12x12 

            d2 = tf.nn.relu(d1)
            d2 = tf.layers.conv2d_transpose(d2, gf_dim*2, [3,3], [2,2], padding="SAME") 
            #d2 = instance_norm(d2, name="d2")
            #d2 = tf.nn.dropout(d2, 0.5)
            d2 = tf.concat([d2, e1], 3)
            # 24x24

            d3 = tf.nn.relu(d2)
            d3 = tf.layers.conv2d_transpose(d3, gf_dim, [3,3], [2,2], padding="SAME") 
            #d3 = instance_norm(d3, name="d3")
            d3 = tf.concat([d3, e0], 3)
            # 48x48

            d7 = tf.nn.relu(d3)
            d7 = tf.layers.conv2d_transpose(d7, 3, [3,3], [2,2], padding="SAME") 
            d7 = tf.nn.tanh(d7)
            # 96x96x3

            conv4 = tf.reshape(d7, [-1, 96*96*3])
            outputs = conv4
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [6]:
class Discriminator:
    def __init__(self, batch_size=32):
        self.reuse = False
        self.X_dim = 96*96*3
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.s_size = 6

    def __call__(self, inputs, training=False, name=''):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            x = tf.reshape(inputs, [-1, 96, 96, 3])
            conv1 = tf.layers.conv2d(x, 64, [4,4], [2,2])
            conv1 = leaky_relu(conv1)
            conv2 = tf.layers.conv2d(conv1, 128, [4,4], [2,2])
            conv2 = leaky_relu(conv2)
            conv3 = tf.layers.conv2d(conv2, 256, [4,4], [2,2])
            conv3 = leaky_relu(conv3)
            conv4 = tf.layers.conv2d(conv3, 512, [4,4], [2,2])
            conv4 = leaky_relu(conv4)
            conv4 = tf.contrib.layers.flatten(conv4)
            fc1 = tf.layers.dense(conv4, 512)
            fc1 = leaky_relu(fc1)
            fc2 = tf.layers.dense(fc1, 256)

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return fc2

In [7]:
# f(x) = ||h(x) - h(x_)|| - ||h(x)||
class Critic(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) - self.h(x_), axis=1) - tf.norm(self.h(x), axis=1)

# f(x) = ||h(x) - h(x_)||
class calc_Norm(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) -self.h(x_))

In [8]:
class GAN:
    def __init__(self, batch_size=32):
        self.batch_size = batch_size
        self.g = Generator_UNet()
        self.d = Discriminator()
        self.z_dim = 100
        self.critic = Critic(self.d)
        self.calcNorm = calc_Norm(self.d)

    def loss(self, Xr, z1, z2):
        Xg = self.g(z1)
        Xg2 = self.g(z2)
        Xg = tf.reshape(Xg, [batch_size, -1])
        Xg2 = tf.reshape(Xg2, [batch_size, -1])

        L_surrogate = tf.reduce_mean(self.critic(Xr, Xg2) - self.critic(Xg, Xg2))
        L_critic = -L_surrogate

        epsilon = tf.random_uniform([], 0.0, 1.0) # 怪しい

        Xhat = epsilon * Xr + (1-epsilon) * Xg
        dhat = self.critic(Xhat, Xg2)

        ddx = tf.gradients(dhat, Xhat)[0]
        ddx = tf.norm(ddx, axis=1)
        ddx = tf.reduce_mean(tf.square(ddx - 1.0) * 10)

        L_critic = L_critic + ddx
        # add each losses to collection
        tf.add_to_collection(
            'g_losses',
            L_surrogate)

        tf.add_to_collection(
            'd_losses',
            L_critic
        )

        return {
            self.g: tf.add_n(tf.get_collection('g_losses'), name='total_g_loss'),
            self.d: tf.add_n(tf.get_collection('d_losses'), name='total_d_loss'),
        }

    def d_train(self, losses, learning_rate=1e-4, beta1=0.5, beta2=0.9):
        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        d_opt_op = d_opt.minimize(losses[self.d], var_list=self.d.variables)
        return d_opt_op

    def g_train(self, losses, learning_rate=1e-4, beta1=0.5, beta2=0.9):
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        g_opt_op = g_opt.minimize(losses[self.g], var_list=self.g.variables)
        return g_opt_op

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs)
        return images

In [ ]:
import numpy as np

X_train = np.load("irasutoya_face_1813x96x96x3_jpg.npy")
X_train = X_train/255
X_dim = 96*96*3
z_dim = 100
batch_size = 32
epochs = 500000
display_epoch = 100
param_save_epoch = 10000
loss = {"dis_loss":[], "gen_loss":[]}

# main func

In [ ]:
p_noise = tf.placeholder(tf.float32, [None, z_dim])
noise_check = np.random.uniform(-1, 1, size=[60, z_dim]).astype(np.float32)

Xr = tf.placeholder(tf.float32, [None, X_dim])
Z1 = tf.placeholder(tf.float32, [None, z_dim])
Z2 = tf.placeholder(tf.float32, [None, z_dim])

gan = GAN()

losses = gan.loss(Xr=Xr, z1=Z1, z2=Z2)
d_train_op = gan.d_train(losses)
g_train_op = gan.g_train(losses)

saver = tf.train.Saver()
#%debug
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        if epoch % display_epoch == 0:
            imgs_gen = gan.sample_images(inputs=noise_check).eval()
            print("saving images")
            save_imgs(imgs_gen, epoch=epoch)

        for _ in range(5):
            # 訓練データを抜粋
            rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)
            X_mb = X_train[rand_index, :].astype(np.float32)
            X_mb = X_mb.reshape([-1, X_dim])

            z1 = np.random.uniform(-1, 1, size=[batch_size, z_dim])
            z2 = np.random.uniform(-1, 1, size=[batch_size, z_dim])

            _, d_loss_value = sess.run([d_train_op, losses[gan.d]], feed_dict={Xr: X_mb, Z1:z1, Z2:z2})


        # 訓練データを抜粋
        rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)
        X_mb = X_train[rand_index, :].astype(np.float32)
        X_mb = X_mb.reshape([-1, X_dim])
        z1 = np.random.uniform(-1, 1, size=[batch_size, z_dim])
        z2 = np.random.uniform(-1, 1, size=[batch_size, z_dim])
        _, g_loss_value, = sess.run([g_train_op, losses[gan.g]], feed_dict={Xr: X_mb, Z1:z1, Z2:z2})

        # 結果をappend
        loss["dis_loss"].append(d_loss_value)
        loss["gen_loss"].append(g_loss_value)
        print("epoch:" + str(epoch))
        # グラフの描画（余裕があったら）
        if epoch % display_epoch == 0:
            save_metrics(loss, epoch)

        if epoch % param_save_epoch == 0:
            path = "model"
            if not os.path.isdir(path):
                os.makedirs(path)

            saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")

called... name of g:test
called... name of g:test
called... name of g:test
saving images
epoch:0
epoch:1
epoch:2
epoch:3
epoch:4
epoch:5
epoch:6
epoch:7
epoch:8
epoch:9
epoch:10
epoch:11
epoch:12
epoch:13
epoch:14
epoch:15
epoch:16
epoch:17
epoch:18
epoch:19
epoch:20
epoch:21
epoch:22
epoch:23
epoch:24
epoch:25
epoch:26
epoch:27
epoch:28
epoch:29
epoch:30
epoch:31
epoch:32
epoch:33
epoch:34
epoch:35
epoch:36
epoch:37
epoch:38
epoch:39
epoch:40
epoch:41
epoch:42
epoch:43
epoch:44
epoch:45
epoch:46
epoch:47
epoch:48
epoch:49
epoch:50
epoch:51
epoch:52
epoch:53
epoch:54
epoch:55
epoch:56
epoch:57
epoch:58
epoch:59
epoch:60
epoch:61
epoch:62
epoch:63
epoch:64
epoch:65
epoch:66
epoch:67
epoch:68
epoch:69
epoch:70
epoch:71
epoch:72
epoch:73
epoch:74
epoch:75
epoch:76
epoch:77
epoch:78
epoch:79
epoch:80
epoch:81
epoch:82
epoch:83
epoch:84
epoch:85
epoch:86
epoch:87
epoch:88
epoch:89
epoch:90
epoch:91
epoch:92
epoch:93
epoch:94
epoch:95
epoch:96
epoch:97
epoch:98
epoch:99
called... name of g:t